In [1]:
import requests
from bs4 import BeautifulSoup
#importing beautiful soap for scrapping the data

## Dependencies

`BBeautifulSoup` :    pip install BeautifulSoup

In [12]:

import pandas as pd
import numpy as np
import itertools #to create efficent looping to fetch more data in a go
import re 
import random 

### Creating BS4 Functions for scrapping

In [52]:
url = "https://www.imdb.com/search/title/?title_type=tv_series&release_date=1980-01-01,2019-12-31&num_votes=80000,&count=200"
#
#https://www.imdb.com/search/title/?title_type=feature&release_date=2020-01-01,2021-12-31&num_votes=20000,&count=200 - 1000 Files done .. processing time 45 min can be used for unlablled
#movie released b/w 2021 and 2020 having votes more than 20k @sanjay 

#https://www.imdb.com/search/title/?title_type=feature&release_date=1980-01-01,2019-12-31&num_votes=500000,&count=200 @mona
#https://www.imdb.com/search/title/?title_type=tv_series&release_date=1980-01-01,2019-12-31&num_votes=80000,&count=200 @mona


#Fetching only 600 movie listing as the processing time is great. once we are good with code we can change the above filter.
def getSoup(url):
    """
    Utility function this get soup function will fetch the above url which stored in url var.
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

def getReviews(soup):
    '''Function returns all reviews including postive and negative..'''
    
    # get a list of user ratings
    user_review_ratings = [tag.previous_element for tag in 
                           soup.find_all('span', attrs={'class': 'point-scale'})]        #can search div by inspect elementor
    
    
    # get the review tags
    user_review_list = soup.find_all('a', attrs={'class':'title'})
    ans = []
    for i in range(5):
        ans.append(user_review_list[random.randint(0, len(user_review_list) -1)])
    links = ["https://www.imdb.com" + tag['href'] for tag in ans]
    return links


def getReviewText(review_url):
    '''Returns the user review text given the review url.'''
    # get the review_url's soup
    soup = getSoup(review_url)
    # find div tags with class text show-more__control
    tag = soup.find('div', attrs={'class': 'text show-more__control'})
    return tag.getText()

def getMovieTitle(review_url):
    '''Returns the movie title from the review url.'''
    # get the review_url's soup
    soup = getSoup(review_url)
    # find h1 tag
    tag = soup.find('h1')
    return list(tag.children)[1].getText()

def getNounChunks(user_review):
    # create the doc object
    doc = nlp(user_review)
    # get a list of noun_chunks
    noun_chunks = list(doc.noun_chunks)
    # convert noun_chunks from span objects to strings, otherwise it won't pickle
    noun_chunks_strlist = [chunk.text for chunk in noun_chunks]
    return noun_chunks_strlist
movies_soup = getSoup(url)

In [53]:
movie_tags = movies_soup.find_all('a', attrs={'class': None})

# filter the a-tags to get just the titles
movie_tags = [tag.attrs['href'] for tag in movie_tags 
              if tag.attrs['href'].startswith('/title') & tag.attrs['href'].endswith('/')]

# remove duplicate links
movie_tags = list(dict.fromkeys(movie_tags))

print("There are a total of " + str(len(movie_tags)) + " movie titles")
print("Displaying 10 titles")
movie_tags[:10]

There are a total of 200 movie titles
Displaying 10 titles


['/title/tt7767422/',
 '/title/tt1520211/',
 '/title/tt0141842/',
 '/title/tt1844624/',
 '/title/tt0098904/',
 '/title/tt0413573/',
 '/title/tt0944947/',
 '/title/tt4270492/',
 '/title/tt4052886/',
 '/title/tt6468322/']

In [54]:
base_url = "https://www.imdb.com"
movie_links = [base_url + tag + 'reviews' for tag in movie_tags]
print("There are a total of " + str(len(movie_links)) + " movie user reviews")
print("Displaying 20 user reviews links")
movie_links[:20]

There are a total of 200 movie user reviews
Displaying 20 user reviews links


['https://www.imdb.com/title/tt7767422/reviews',
 'https://www.imdb.com/title/tt1520211/reviews',
 'https://www.imdb.com/title/tt0141842/reviews',
 'https://www.imdb.com/title/tt1844624/reviews',
 'https://www.imdb.com/title/tt0098904/reviews',
 'https://www.imdb.com/title/tt0413573/reviews',
 'https://www.imdb.com/title/tt0944947/reviews',
 'https://www.imdb.com/title/tt4270492/reviews',
 'https://www.imdb.com/title/tt4052886/reviews',
 'https://www.imdb.com/title/tt6468322/reviews',
 'https://www.imdb.com/title/tt0364845/reviews',
 'https://www.imdb.com/title/tt2442560/reviews',
 'https://www.imdb.com/title/tt2788432/reviews',
 'https://www.imdb.com/title/tt4574334/reviews',
 'https://www.imdb.com/title/tt1043813/reviews',
 'https://www.imdb.com/title/tt0903747/reviews',
 'https://www.imdb.com/title/tt0203259/reviews',
 'https://www.imdb.com/title/tt0386676/reviews',
 'https://www.imdb.com/title/tt7335184/reviews',
 'https://www.imdb.com/title/tt0108778/reviews']

In [55]:
movie_soups = [getSoup(link) for link in movie_links]

# get all 500 movie review links
movie_review_list = [getReviews(movie_soup) for movie_soup in movie_soups]

#movie_review_list = list(itertools.chain(*movie_review_list))
#print(len(movie_review_list))

#print("There are a total of " + str(len(movie_review_list)) + " individual movie reviews")
#print("Displaying 10 reviews")
#movie_review_list[:10]

In [58]:

movie_review_list = list(itertools.chain(*movie_review_list))
print(len(movie_review_list))

print("There are a total of " + str(len(movie_review_list)) + " individual movie reviews")
print("Displaying 10 reviews")
movie_review_list[:10]

1000
There are a total of 1000 individual movie reviews
Displaying 10 reviews


['https://www.imdb.com/review/rw6788259/',
 'https://www.imdb.com/review/rw5546069/',
 'https://www.imdb.com/review/rw5494767/',
 'https://www.imdb.com/review/rw5494767/',
 'https://www.imdb.com/review/rw7423572/',
 'https://www.imdb.com/review/rw2412620/',
 'https://www.imdb.com/review/rw4133954/',
 'https://www.imdb.com/review/rw2470759/',
 'https://www.imdb.com/review/rw2714432/',
 'https://www.imdb.com/review/rw2714432/']

In [59]:
review_texts = [getReviewText(url) for url in movie_review_list]

# get movie name from the review link
movie_titles = [getMovieTitle(url) for url in movie_review_list]

# label each review with negative or positive

# construct a dataframe
df = pd.DataFrame({
             'user_review': review_texts })

KeyboardInterrupt: 

In [46]:
df.head(5) #converting it into data frama

,user_review
0,"Simply,this is the best Bond since Connery 's ..."
1,After a few disappointing films it looked like...
2,"Everything about the latest James Bond flick, ..."
3,I have always said that the James Bond franchi...
4,"Don't get me wrong I really like James Bond, a..."


In [47]:
text_list = [m for m in df['user_review']]
#text_list

In [48]:
text_list_length = [len(m.split()) for m in text_list]     
df['length'] = text_list_length
df

,user_review,length
0,"Simply,this is the best Bond since Connery 's ...",420
1,After a few disappointing films it looked like...,434
2,"Everything about the latest James Bond flick, ...",246
3,I have always said that the James Bond franchi...,167
4,"Don't get me wrong I really like James Bond, a...",341
...,...,...
995,I watched this soon after it was released on v...,216
996,This is the One that Launched the Careers of a...,299
997,"Lock, Stock and Two Smoking Barrels is set in ...",798
998,"From debuting director Guy Ritchie, the man wh...",291


In [60]:
df = df[df['length'] < 250]
df

,user_review,length
2,"Everything about the latest James Bond flick, ...",246
3,I have always said that the James Bond franchi...,167
5,"""Skyfall"" is a good film and it made a buttloa...",191
7,"I'm not an expert on the Bond canon, but I lov...",241
9,An unknown bad guy (Javier Bardem) has obtaine...,200
...,...,...
992,This film is about an elderly man going on an ...,160
993,"Pixar's ""Up"" is a colorful animation adventure...",227
994,A sure instant classic from Pixar. From the mi...,228
995,I watched this soon after it was released on v...,216


In [61]:
df.drop('length', axis=1, inplace=True)
df

,user_review
2,"Everything about the latest James Bond flick, ..."
3,I have always said that the James Bond franchi...
5,"""Skyfall"" is a good film and it made a buttloa..."
7,"I'm not an expert on the Bond canon, but I lov..."
9,An unknown bad guy (Javier Bardem) has obtaine...
...,...
992,This film is about an elderly man going on an ...
993,"Pixar's ""Up"" is a colorful animation adventure..."
994,A sure instant classic from Pixar. From the mi...
995,I watched this soon after it was released on v...


In [62]:
#converting only reviews to CSV & removing the index
df.to_csv('data_scrapped/data.csv', index=False) 

In [57]:
import csv

with open("data_scrapped/data.csv", "r") as f:
        reader = csv.reader(f)
        rownumber = 447
        for row in reader:
             g=open(str(rownumber)+".txt","w")
             g.write(str(row))
             rownumber = rownumber + 1
             g.close()